# Inspecting the model

In this notebook we will present some techniques to log and visualize the model behaviour during training. Neural networks have been widely critized because of the lack of interpretation of their internal parameters.

The lack of interpretability leads, among other thigns, to make neural models error prone. While this is true, we still have some tools to try to debug our network and to understand what the model is doing.

In [1]:
import numpy as np
import tensorflow as tf

In [106]:
# Load the dataset into a numpy keyed structure
newsgroups = np.load('./resources/newsgroup.npz')

# Define the batch size and the number of labels
batch_size = 100
num_classes = newsgroups['labels'].shape[0]

def dataset_input_fn(dataset):
    """
    Creates an input function using the `numpy_input_fn` method from
    tensorflow, based on the dataset we want to use.
    
    Args:
        dataset: String that represents the dataset (should be `train` or `test`)
    
    Returns:
        An `numpy_input_fn` function to feed to an estimator
    """
    assert dataset in ('train', 'test'), "The selected dataset should be `train` or `test`"
    
    return tf.estimator.inputs.numpy_input_fn(
        x={'input_data': newsgroups['%s_data' % dataset]},
        y=newsgroups['%s_target' % dataset],
        batch_size=batch_size,
        num_epochs=1 if dataset == 'test' else None,
        shuffle=dataset == 'train'
    )

### The easiest way of logging values

If you only need to see some numerical values during training, you can print them in the console (or notebook in this case).

To add any operation that is performed inside the training cycle, the `Estimator.train` method provides hooks. Hooks, which are formally instances of subclasses of `SessionRunHook`, will be called after each epoch **TODO check this** to perform the operation you want, depending on the type of hook. In this particulaer case, the `LoggingTensorHook` will print in console the tensors we give as parameters, and we can personalize after how many iterations the print will occur. This will also work for the evaluate and predict methods.

To try the logging, just run the above training phase with the model we presented on the previous notebook.

In [ ]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {'probabilities': 'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
mlp_classifier.train(
    input_fn=dataset_input_fn('train'),
    steps=2000,
    hooks=[logging_hook]
)

## Tensorboard

There is a limit to what we can print and interpret on console. Tensorflow comes with its own (and very complete) visualization tool: Tensorboard. In the rest of this tutorial, we will explain how to use Tensorboard to log scalar values like metrics of performance, histogram values like the activation of the cells in each network layer. In the next notebook we will see how to plot and inspect embeddings to show how the document embeddings relate to each other.

Tensorboard is based on operations called summaries which record the tensor variable to log. Unlike the previous example, summaries, as all operations, must be compiled along with the model in order to be included in the execution graph. There is a summary operation for each type of data that we want to log: scalars, tensors (histogram or tensor), audio, images and text.

In any tensorflow code where we want to save variables for Tensorboard, we have to add some code with the following structure:

```
    # The definition of your variables
    ...
    # The summary operations
    tf.summary.histogram('softmax_tensor', probabilities_tensor)
    tf.summary.scalar('loss', loss_value)
    
    # The merge operation
    tf.summary.merge_all()
    
    # The write operation
    ...
```

The `summary.histogram` and `summary.scalar` will evaluate the value of the variable at that point during the execution of the graph. Then, the `summary.merge_all` takes all the summary operations added up to that moment and creates a single output with all the information, so the result can be written to disk only once.

Now, for older versions of tensorflow or if you are not using Estimators, the write operation uses the `summary.FileWriter` class to write your data. On the other hand, the Estimator wraps this task into a special Hook for summary operations called `SummarySaverHook`.

In the following cell we have the same model structure as before (with less comments) and we add the summary operations to the graph, and finally the summary hook.

In [131]:
def build_model(input_data, mode):
    """Creates the model layers.
    
    Args:
        input_data: a Tensor with shape [batch_size, feature_size]
    
    Returns:
        The logits of the output layer."""
    hidden1 = tf.layers.dense(inputs=input_data, units=250, activation=tf.nn.relu,
                              name='hidden_layer_1')
    hidden2 = tf.layers.dense(inputs=hidden1, units=100, activation=tf.nn.relu,
                              name='hidden_layer_2')
    dropout = tf.layers.dropout(inputs=hidden2, rate=0.4,
                                training=(mode == tf.estimator.ModeKeys.TRAIN))
    logits = tf.layers.dense(inputs=dropout, units=num_classes, name='logits')

    return (logits)

def mlp_model_fn(features, labels, mode):
    """Model function for MLP.
    
    Args:
        features: a dictionary where the values are input tensors with shape
            [batch_size, feature_size]
        labels: a tensor with shape [batch_size]
        mode: a constant, one of `tf.estimator.ModeKeys.`
    
    Returns:
        An instance of ´tf.estimator.EstimatorSpec´.
    """
    logits = build_model(features['input_data'], mode)

    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=num_classes)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    accuracy_op = tf.metrics.accuracy(labels=labels, predictions=predictions['classes'], name='accuracy')
    # Add the summary operation to log the tensor with the predictions
    tf.summary.histogram('softmax_tensor', predictions['probabilities'])
    # Add the summary operation to log the value of the accuracy
    tf.summary.scalar('accuracy', accuracy_op[1])
    summary_op = tf.summary.merge_all()
    summary_hook = tf.train.SummarySaverHook(save_steps=100, summary_op=summary_op)

    if mode == tf.estimator.ModeKeys.TRAIN:
        
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op,
                                          training_hooks=[summary_hook])

    eval_metric_ops = {'accuracy': accuracy_op}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops,
                                      evaluation_hooks=[summary_hook])

Now we can create the Estimator as before with the summary operations compiled into the graph. Note that, as we have a different graph, we have to use a new model_dir or it would fail when loading the previous checkpoint.

In [124]:
mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn, model_dir='20news_mlp_model_summaries')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_tf_random_seed': None, '_service': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcc10669550>, '_model_dir': '20news_mlp_model_summaries', '_is_chief': True, '_num_worker_replicas': 1}


In [125]:
mlp_classifier.train(input_fn=dataset_input_fn('train'), steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into 20news_mlp_model_summaries/model.ckpt.
INFO:tensorflow:loss = 3.02155637741, step = 1
INFO:tensorflow:global_step/sec: 225.826
INFO:tensorflow:loss = 1.49042844772, step = 101 (0.444 sec)
INFO:tensorflow:global_step/sec: 252.165
INFO:tensorflow:loss = 0.842517077923, step = 201 (0.397 sec)
INFO:tensorflow:global_step/sec: 256.659
INFO:tensorflow:loss = 1.1786583662, step = 301 (0.390 sec)
INFO:tensorflow:global_step/sec: 245.672
INFO:tensorflow:loss = 1.20692622662, step = 401 (0.407 sec)
INFO:tensorflow:global_step/sec: 247.319
INFO:tensorflow:loss = 0.765724778175, step = 501 (0.404 sec)
INFO:tensorflow:global_step/sec: 253.92
INFO:tensorflow:loss = 0.696938157082, step = 601 (0.394 sec)
INFO:tensorflow:global_step/sec: 244.71
INFO:tensorflow:loss = 0.848085165024, step = 701 (0.409 sec)
INFO:tensorflow:global_step/sec: 242.592
INFO:tensorflow:loss = 0.772474586964, step = 801 (0.412 sec)
INFO:t

In [126]:
mlp_classifier.evaluate(input_fn=dataset_input_fn('test'))

INFO:tensorflow:Starting evaluation at 2017-11-14-18:25:17
INFO:tensorflow:Restoring parameters from 20news_mlp_model_summaries/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-11-14-18:25:17
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.742698, global_step = 2000, loss = 0.948799


{'accuracy': 0.74269783, 'global_step': 2000, 'loss': 0.94879884}

The next thing to do is to go to the Tensorboard dashboard in the model directory and inspect the obtained values.

## Multiple iterations

If we run the same experiment as before several times, the model will be restored from the last checkpoint and the training will recommence using the current weights and biases. Even more, if you don't use Estimators the results of different runs are stored in the same folder and coexists in a mess of metric values.

We actually want to compare several runs of the same experiment, or perhaps compare the performance of several classifiers in the same graph. For that, we will change the structure of the directories and Tensorboard will organize and show the results accordingly. We will use an experiment counter to keep track of how many iterations we have done.

In [129]:
EXPERIMENT_COUNTER = 0

In [133]:
import os

mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn,
    model_dir=os.path.join('20news_mlp_model_summaries', 'iter{}'.format(EXPERIMENT_COUNTER)))
mlp_classifier.train(input_fn=dataset_input_fn('train'), steps=2000)
EXPERIMENT_COUNTER += 1

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_tf_random_seed': None, '_service': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcc18369828>, '_model_dir': '20news_mlp_model_summaries/iter2', '_is_chief': True, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into 20news_mlp_model_summaries/iter2/model.ckpt.
INFO:tensorflow:loss = 3.1666135788, step = 1
INFO:tensorflow:global_step/sec: 219.791
INFO:tensorflow:loss = 1.21929335594, step = 101 (0.456 sec)
INFO:tensorflow:global_step/sec: 233.729
INFO:tensorflow:loss = 1.10630130768, step = 201 (0.428 sec)
INFO:tensorflow:glob

## Contrib: Monitoring performance while training

We know so far how to visualize the metrics related to training, but it is more interesting to compare the performance of the classifier in the validation dataset. Furthermore, we would like to stop the training if the validation performance drops too much.

In the contrib.learn module of tensorflow we found an `Experiment` class that will run the train and evaluation cycle for us. Even if this not hard to implement using a for loop, we recommend to use the functions provided by tensorflow as they support training on multiple servers.

In [128]:
mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn,
    model_dir=os.path.join('20news_mlp_model_summaries', 'iter{}'.format(EXPERIMENT_COUNTER)))

experiment = tf.contrib.learn.Experiment(
    mlp_classifier,
    train_input_fn=train_input_fn,
    eval_input_fn=eval_input_fn,
    train_steps=1000,
    train_steps_per_iteration=100
    )
experiment.continuous_train_and_eval()

EXPERIMENT_COUNTER += 1

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_tf_random_seed': None, '_service': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcbe8c527f0>, '_model_dir': '20news_mlp_model_summaries/iter22', '_is_chief': True, '_num_worker_replicas': 1}
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into 20news_mlp_model_summaries/iter22/model.ckpt.
INFO:tensorflow:loss = 3.02090978622, step = 1
INFO:tensorflow:Saving checkpoints for 100 into 20news_mlp_model_summaries/iter22/model.ckpt.
INFO:tensorflow:Loss for final step: 2.65106534958.
INFO:tensorflow:Evaluating mod

INFO:tensorflow:Loss for final step: 1.46542322636.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2017-11-14-18:37:06
INFO:tensorflow:Restoring parameters from 20news_mlp_model_summaries/iter22/model.ckpt-900
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Finished evaluation at 2017-11-14-18:37:06
INFO:tensorflow:Saving dict for global step 900: accuracy = 0.655, global_step = 900, loss = 1.10596
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from 20news_mlp_model_summaries/iter22/model.ckpt-900
INFO:tensorflow:Saving checkpoints for 901 into 20news_mlp_model_summaries/iter22/model.ckpt.
INFO:tensorflow:loss = 1.77530324459, step = 901
INFO:tensorflow:Saving checkpoints for 1000 into 20news_mlp_model_summaries/iter22/model.ckpt.
INFO:tensorflow:Loss for final step: 1.323315382.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting

# Activity

 1. Use Tensoboard to visualize, the precision and recall of several runs and several models in the same graph.
 2. Create an early stop training cycle using the `contrib.learn.Experiment` class or the `tf.estimator.train_and_evaluate` function. This cycle must train the model until the performance on the test dataset drops.